In [58]:
import tensorflow as tf
with tf.device('/GPU:0'):
    import pygad
    import numpy as np

    class GreenshieldsModel:
        def __init__(self, free_flow_speed=40.0, jam_density=120.0):
            """
            初始化格林希尔兹模型。
            
            :param free_flow_speed: 自由流速度 (km/h)，默认40 km/h。
            :param jam_density: 阻塞密度 (辆/km)，默认120辆/km。
            """
            self.vf = free_flow_speed
            self.kj = jam_density

        def calculate_speed(self, density):
            """
            使用格林希尔兹模型计算速度。
            
            :param density: 车流密度 (辆/km)。
            :return: 速度 (km/h)。
            """
            if density > self.kj:
                return 0.0
            return self.vf * (1 - density / self.kj)

    class Intersection:
        def __init__(self, flows, saturation_flows, phase_map, lost_time=10):
            """
            初始化交叉口。
            
            :param flows: 12个每小时车流量 (辆/小时)，顺序为[东左, 东右, 东直, 西左, 西右, 西直, 南左, 南右, 南直, 北左, 北右, 北直]。
            :param saturation_flows: 12个饱和流量 (辆/小时)，同上顺序。
            :param phase_map: 12个整数，映射每个运动到相位 (0-3)。
            :param lost_time: 每周期失时 (秒)，默认10秒。
            """
            self.flows = np.array(flows)
            self.saturation_flows = np.array(saturation_flows)
            self.phase_map = np.array(phase_map)
            self.num_phases = len(set(phase_map))
            self.lost_time = lost_time

        def calculate_total_delay(self, green_times):
            """
            计算交叉口总延迟。
            
            :param green_times: 每个相位的绿灯时间 (秒)。
            :return: 总延迟 (车辆-秒/小时)。
            """
            if len(green_times) != self.num_phases:
                return float('inf')
            green_times = np.array(green_times)
            if np.any(green_times < 10):
                return float('inf')
            sum_g = np.sum(green_times)
            if sum_g < 80 or sum_g > 110:  # 修改为80-110秒
                return float('inf')
            C = sum_g + self.lost_time
            total_delay = 0
            for m in range(len(self.flows)):
                j = self.phase_map[m]
                g_j = green_times[j]
                λ_m = g_j / C
                q_m = self.flows[m]
                s_m = self.saturation_flows[m]
                Y_m = q_m / s_m
                if λ_m * Y_m >= 1:
                    d_m = float('inf')
                else:
                    d_m = (C / 2) * ((1 - λ_m)**2) / (1 - λ_m * Y_m)
                if np.isinf(d_m):
                    total_delay += 1e6
                else:
                    total_delay += q_m * d_m
            return total_delay

    def optimize_signals(intersection, num_generations=100, sol_per_pop=50):
        """
        使用遗传算法优化信号灯配置。
        
        :param intersection: Intersection对象。
        :param num_generations: 遗传算法代数，默认100。
        :param sol_per_pop: 种群大小，默认50。
        :return: 优化的绿灯时间和最小总延迟。
        """
        num_parents = 20
        num_genes = intersection.num_phases
        init_range_low = 20
        init_range_high = 50

        def fitness_func(ga_instance, solution, sol_idx):
            delay = intersection.calculate_total_delay(solution)
            return -delay if not np.isinf(delay) else -1e6

        ga_instance = pygad.GA(
            num_generations=num_generations,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_parents_mating=num_parents,
            num_genes=num_genes,
            gene_type=float,
            init_range_low=init_range_low,
            init_range_high=init_range_high,
            parent_selection_type="sss",
            crossover_type="single_point",
            mutation_type="random",
            mutation_percent_genes=10
        )

        ga_instance.run()
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        min_delay = -solution_fitness
        return solution, min_delay
    #     # 示例使用
    # if __name__ == "__main__":
    #     # 工作日高峰时段车流量（每小时平均，假设总计72小时高峰）
    #     flows_per_hour = [
    #         13323.42 / 72, 68097.48 / 72, 66617.1 / 72,  # 方向1：东左、东右、东直
    #         4133.7 / 72, 21127.8 / 72, 20668.5 / 72,     # 方向2：西左、西右、西直
    #         9983.07 / 72, 51024.58 / 72, 49915.35 / 72,  # 方向3：南左、南右、南直
    #         15132.24 / 72, 77342.56 / 72, 75661.2 / 72   # 方向4：北左、北右、北直
    #     ]
    #     saturation_flows = [1200, 1800, 1800] * 4  # 左转1200，右转/直行1800
    #     phase_map = [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0]  # 相位映射

    #     # 创建交叉口对象
    #     intersection = Intersection(flows_per_hour, saturation_flows, phase_map)

    #     # 优化信号灯
    #     optimal_greens, min_delay = optimize_signals(intersection)

    #     print("优化的绿灯时间:", optimal_greens)
    #     print("最小总延迟:", min_delay)

    #     # 示例格林希尔兹模型使用
    #     greenshields = GreenshieldsModel()
    #     density = 60  # 示例密度
    #     speed = greenshields.calculate_speed(density)
    #     print(f"密度 {density} 辆/km 时的速度: {speed} km/h")



In [59]:
import os
import sys

# 先导入 TensorFlow
import tensorflow as tf
import torch
print(torch.__version__)

# 检查 TensorFlow 版本
print("TensorFlow version:", tf.__version__)
print(sys.executable)
# 检查是否有可用的 GPU
try:
    tf_gpus = tf.config.list_physical_devices('GPU')
    if tf_gpus:
        print("GPUs available for TensorFlow:")
        for gpu in tf_gpus:
            print(gpu)
    else:
        print("No GPUs available for TensorFlow.")
except Exception as e:
    print("TensorFlow GPU check failed:", e)

# 再导入 PyTorch
try:
    import torch
    print("PyTorch version:", torch.__version__)
    # 检查是否有可用的 GPU
    torch_gpus = torch.cuda.device_count()
    if torch_gpus > 0:
        print(f"{torch_gpus} GPUs available for PyTorch:")
        for idx in range(torch_gpus):
            print(f"GPU {idx}: {torch.cuda.get_device_name(idx)}")
    else:
        print("No GPUs available for PyTorch.")

    # 检查cuda版本
    cuda_version = torch.version.cuda
    if cuda_version:
        print("CUDA version:", cuda_version)
    # 检查cuDNN版本
    cudnn_version = torch.backends.cudnn.version()
    if cudnn_version:
        print("cuDNN version:", cudnn_version)
except Exception as e:
    print("PyTorch GPU check failed:", e)

# 检查是否有可用的 GPU
if not tf_gpus and torch_gpus == 0:
    print("No GPUs available for both TensorFlow and PyTorch.")
    raise RuntimeError("No GPUs available for both TensorFlow and PyTorch.")

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 设置可见的 GPU 设备
# 运行 TensorFlow 和 PyTorch 的简单测试
try:
    # TensorFlow 测试
    a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
    c = tf.matmul(a, b)
    print("TensorFlow matrix multiplication result:\n", c.numpy())
    # PyTorch 测试
    a_torch = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
    b_torch = torch.tensor([[5.0, 6.0], [7.0, 8.0]])
    c_torch = torch.matmul(a_torch, b_torch)
    print("PyTorch matrix multiplication result:\n", c_torch.numpy())
except Exception as e:
    print("Error during TensorFlow or PyTorch test:", e)
    raise

# 检查当前gpu型号
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU found.")

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("No GPU found.")

1.10.1+cu113
TensorFlow version: 2.10.0
c:\Users\hhui8\.conda\envs\Mathmatical\python.exe
GPUs available for TensorFlow:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PyTorch version: 1.10.1+cu113
1 GPUs available for PyTorch:
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA version: 11.3
cuDNN version: 8200
TensorFlow matrix multiplication result:
 [[19. 22.]
 [43. 50.]]
PyTorch matrix multiplication result:
 [[19. 22.]
 [43. 50.]]
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
import os
import pandas as pd
import numpy as np
import pygad
import tensorflow as tf

# 使用GPU加速（如果可用）
with tf.device('/GPU:0'):
    class GreenshieldsModel:
        def __init__(self, free_flow_speed=40.0, jam_density=120.0):
            self.vf = free_flow_speed
            self.kj = jam_density

        def calculate_speed(self, density):
            if density > self.kj:
                return 0.0
            return self.vf * (1 - density / self.kj)

    class Intersection:
        def __init__(self, flows, saturation_flows, phase_map, lost_time=10):
            self.flows = np.array(flows)
            self.saturation_flows = np.array(saturation_flows)
            self.phase_map = np.array(phase_map)
            self.num_phases = len(set(phase_map))
            self.lost_time = lost_time

        def calculate_total_delay(self, green_times):
            """
            计算交叉口总延迟。
            
            :param green_times: 每个相位的绿灯时间 (秒)。
            :return: 总延迟 (车辆-秒/小时)。
            """
            if len(green_times) != self.num_phases:
                return float('inf')
            green_times = np.array(green_times)
            if np.any(green_times < 10):
                return float('inf')
            sum_g = np.sum(green_times)
            if sum_g < 50 or sum_g > 170:
                return float('inf')
            C = sum_g + self.lost_time

            # 使用向量化操作替代循环
            # 获取每个车流对应的相位绿灯时间
            phase_green_times = green_times[self.phase_map]
            # 计算绿灯时间占比
            lambda_m = phase_green_times / C
            # 计算流量比
            Y_m = self.flows / self.saturation_flows
            # 计算延迟
            valid_mask = (lambda_m * Y_m) < 1
            d_m = np.empty_like(self.flows, dtype=float)
            d_m[valid_mask] = (C / 2) * ((1 - lambda_m[valid_mask])**2) / (1 - lambda_m[valid_mask] * Y_m[valid_mask])
            d_m[~valid_mask] = float('inf')
            # 计算总延迟
            total_delay = np.sum(self.flows * d_m)
            # 处理无穷大延迟
            if np.isinf(total_delay):
                return 1e6 * np.sum(self.flows[~valid_mask]) + np.sum(self.flows[valid_mask] * d_m[valid_mask])
            return total_delay
       # 修复GreenWaveCoordinator类
    class GreenWaveCoordinator:
        def __init__(self, intersections, distances):
            self.intersections = intersections
            self.distances = distances
            self.common_cycle = None
            self.phase_differences = None

        def calculate_common_cycle(self):
            # 基于所有交叉口的最优周期，取一个折中的公共周期
            # 这里我们使用一个合理的默认值作为临时解决方案
            self.common_cycle = 110  # 假设公共周期为120秒
            return self.common_cycle

        def calculate_phase_differences(self, speed=30):
            if self.common_cycle is None:
                self.calculate_common_cycle()

            # 转换速度为米/秒
            speed_mps = speed * 1000 / 3600
            self.phase_differences = []
            cumulative_distance = 0

            for i, distance in enumerate(self.distances):
                cumulative_distance += distance
                # 计算理想相位差(秒)
                ideal_diff = cumulative_distance / speed_mps
                # 转换为周期的百分比
                phase_diff = (ideal_diff / self.common_cycle) * 100
                self.phase_differences.append(phase_diff)

            return self.phase_differences

        def coordinate_signals(self):
            # 计算公共周期和相位差
            self.calculate_common_cycle()
            self.calculate_phase_differences()

            # 基于公共周期和相位差优化每个交叉口的信号灯配时
            optimized_results = []
            for i, intersection in enumerate(self.intersections):
                # 调用optimize_signals函数，限制周期为公共周期
                result = optimize_signals(intersection, fixed_cycle=self.common_cycle)
                optimized_results.append(result)

            return optimized_results
    # 优化signals函数修改
    def optimize_signals(intersection, num_generations=50, sol_per_pop=30, early_stopping=5, fixed_cycle=None):
        num_parents = 10
        num_genes = intersection.num_phases
        init_range_low = 20
        init_range_high = 50

        # 添加早停机制变量
        best_fitness = -float('inf')
        no_improvement_count = 0

        def fitness_func(ga_instance, solution, sol_idx):
            delay = intersection.calculate_total_delay(solution)
            # 如果有固定周期限制
            if fixed_cycle is not None:
                sum_g = np.sum(solution)
                # 惩罚偏离固定周期的解
                cycle_penalty = abs(sum_g + intersection.lost_time - fixed_cycle) * 10
                delay += cycle_penalty
            return -delay if not np.isinf(delay) else -1e6

        # 添加早停回调函数
        def callback_generation(ga_instance):
            nonlocal best_fitness, no_improvement_count
            current_best_fitness = ga_instance.best_solution()[1]

            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                no_improvement_count = 0
            else:
                no_improvement_count += 1

            if no_improvement_count >= early_stopping:
                print(f"早停机制触发，在第{ga_instance.generations_completed+1}代停止")
                return "stop"

        ga_instance = pygad.GA(
            num_generations=num_generations,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_parents_mating=num_parents,
            num_genes=num_genes,
            gene_type=float,
            init_range_low=init_range_low,
            init_range_high=init_range_high,
            parent_selection_type="sss",
            crossover_type="single_point",
            mutation_type="random",
            mutation_num_genes=1,  # 直接设置要突变的基因数量，避免警告
            on_generation=callback_generation  # 使用回调函数实现早停
        )

        ga_instance.run()  # 使用标准run方法

        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        min_delay = -solution_fitness
        return solution, min_delay, np.sum(solution) + intersection.lost_time  # 返回周期
   
    def dynamic_flow_adjustment(intersection_name, is_weekday, hour):
        """
        根据时段动态调整流量
        :param intersection_name: 交叉口名称
        :param is_weekday: 是否为工作日
        :param hour: 小时(0-23)
        :return: 调整后的流量
        """
        # 读取基础流量数据
        file_name = f"{intersection_name}_{'工作日' if is_weekday else '节假日'}分方向分时段车流量_带聚类标签.csv"
        file_path = os.path.join('问题2支撑材料', file_name)

        try:
            df = pd.read_csv(file_path, encoding='gbk')
        except:
            df = pd.read_csv(file_path, encoding='utf-8')

        # 提取方向列
        direction_keywords = ['东', '南', '西', '北', '方向']
        direction_columns = [col for col in df.columns if any(key in str(col) for key in direction_keywords)][:4]

        # 假设数据按小时排序，这里简单处理
        # 实际应用中应该有时间列用于匹配
        hour_index = min(hour, len(df)-1)
        flows = df.iloc[hour_index][direction_columns].values

        # 按1:2:2比例分配左、右、直行车流量
        split_ratio = np.array([1/5, 2/5, 2/5])
        flows_per_hour = np.repeat(flows, 3) * np.tile(split_ratio, len(flows))

        # 确保有12个流量值
        flows_per_hour = flows_per_hour[:12]
        if len(flows_per_hour) < 12:
            flows_per_hour = np.pad(flows_per_hour, (0, 12 - len(flows_per_hour)), 'constant')

        return flows_per_hour.tolist()
    # 解决中文乱码问题
    pd.set_option('display.unicode.east_asian_width', True)

    # 定义路径和文件列表
    DATA_DIR = '问题2支撑材料'
    csv_files = [
        '环东路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '环东路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '环西路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '环西路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '经二路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '经二路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '经三路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '经三路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '经四路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '经四路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '经五路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '经五路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '经一路-纬中路_工作日分方向分时段车流量_带聚类标签.csv',
        '经一路-纬中路_节假日分方向分时段车流量_带聚类标签.csv',
        '经中路-环北路_工作日分方向分时段车流量_带聚类标签.csv',
        '经中路-环北路_节假日分方向分时段车流量_带聚类标签.csv',
        '经中路-环南路_工作日分方向分时段车流量_带聚类标签.csv',
        '经中路-环南路_节假日分方向分时段车流量_带聚类标签.csv',
        '经中路-纬一路_工作日分方向分时段车流量_带聚类标签.csv',
        '经中路-纬一路_节假日分方向分时段车流量_带聚类标签.csv',
        '纬中路-景区出入口_工作日分方向分时段车流量_带聚类标签.csv',
        '纬中路-景区出入口_节假日分方向分时段车流量_带聚类标签.csv'
    ]

    # 定义每个路口的相位映射
    phase_maps = {
        '环东路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '环西路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经二路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经三路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经四路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经五路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经一路-纬中路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经中路-环北路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经中路-环南路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '经中路-纬一路': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0],
        '纬中路-景区出入口': [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0]
    }

    # 定义饱和流量（左转1200，右转/直行1800）
    SATURATION_FLOWS = [1200, 1800, 1800] * 4

        # 处理单个CSV文件并返回每小时车流量
    def process_csv(file_path, intersection_name, is_weekday):
        try:
            # 直接使用gbk编码
            try:
                df = pd.read_csv(file_path, encoding='gbk')
            except:
                df = pd.read_csv(file_path, encoding='utf-8')

            if df is None:
                print(f"无法读取文件: {file_path}")
                return None

            # 提取方向列
            direction_keywords = ['东', '南', '西', '北', '方向']
            direction_columns = [col for col in df.columns if any(key in str(col) for key in direction_keywords)]

            # 确保有4个方向列
            if len(direction_columns) < 4:
                if len(direction_columns) == 0:
                    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
                    direction_columns = numeric_cols[:4]
                    while len(direction_columns) < 4:
                        direction_columns.append(f'填充列_{len(direction_columns)}')
                        df[direction_columns[-1]] = 0
                else:
                    while len(direction_columns) < 4:
                        direction_columns.append(direction_columns[-1])
            else:
                direction_columns = direction_columns[:4]

            # 提取时段信息（假设CSV中有'时段'列）
            time_period = '未知时段'
            if '时段' in df.columns:
                # 获取最常见的时段值
                time_period = df['时段'].mode().iloc[0] if not df['时段'].empty else '未知时段'
            elif '聚类标签' in df.columns:
                # 根据聚类标签映射到时段
                cluster_to_period = {0: '高峰', 1: '平峰', 2: '低峰'}
                cluster = df['聚类标签'].mode().iloc[0] if not df['聚类标签'].empty else 3
                time_period = cluster_to_period.get(cluster, '未知时段')

            # 向量化计算每小时车流量
            avg_flows = df[direction_columns].mean().values
            # 按1:2:2比例分配左、右、直行车流量
            split_ratio = np.array([1/5, 2/5, 2/5])
            flows_per_hour = np.repeat(avg_flows, 3) * np.tile(split_ratio, len(avg_flows))

            # 确保有12个流量值
            flows_per_hour = flows_per_hour[:12]
            if len(flows_per_hour) < 12:
                flows_per_hour = np.pad(flows_per_hour, (0, 12 - len(flows_per_hour)), 'constant')

            return {
                'intersection_name': intersection_name,
                'is_weekday': is_weekday,
                'time_period': time_period,
                'flows': flows_per_hour.tolist()
            }
        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {e}")
            return None

        # 在main函数前添加
    import concurrent.futures

     # 修改main函数，支持绿波协调和动态流量调整
    def main(use_green_wave=True, dynamic_adjustment=True):
        results = []
        data_list = []

        # 预先加载所有数据
        for file in csv_files:
            parts = file.split('_')
            intersection_name = parts[0]
            is_weekday = '工作日' in parts[1]
            file_path = os.path.join(DATA_DIR, file)
            data = process_csv(file_path, intersection_name, is_weekday)
            if data is not None:
                phase_map = phase_maps.get(intersection_name, [3, 1, 1, 3, 1, 1, 2, 0, 0, 2, 0, 0])
                data_list.append((data, phase_map))

        # 如果使用绿波协调
        if use_green_wave:
            # 假设这是12个交叉口的顺序和间距
            # 实际应用中需要根据实际道路网络定义
            intersection_order = ["经一路-纬中路", "经二路-纬中路", "经三路-纬中路",
                                 "经四路-纬中路", "经五路-纬中路", "环东路-纬中路",
                                 "环西路-纬中路", "经中路-环北路", "经中路-环南路",
                                 "经中路-纬一路", "纬中路-景区出入口", "新增交叉口"]
            distances = [300, 250, 320, 280, 350, 400, 450, 380, 320, 290, 310, 270]  # 假设的间距(米)

            # 创建绿波协调器
            coordinator = GreenWaveCoordinator(
                [Intersection(data['flows'], SATURATION_FLOWS, phase_map) for data, phase_map in data_list],
                distances
            )

            # 进行绿波协调优化
            coordinated_results = coordinator.coordinate_signals()

                        # 收集协调后的结果
        for i, (future, data) in enumerate(zip(coordinated_results, data_list)):
            try:
                optimal_greens, min_delay, cycle = future
                # 对最优绿灯时间和最小延迟进行四舍五入取整
                rounded_greens = [round(green) for green in optimal_greens]
                rounded_delay = round(min_delay)
                
                # 构建结果字典，将绿灯时长按相位分开
                result = {
                    '交叉口': data[0]['intersection_name'],
                    '是否为工作日': data[0]['is_weekday'],
                    '时段': data[0]['time_period'],
                    '最小延迟': rounded_delay,
                    '周期': round(cycle),
                    '相位差': round(coordinator.phase_differences[i] if i < len(coordinator.phase_differences) else 0)
                }
                
                # 添加各相位绿灯时长
                for phase_idx, green_time in enumerate(rounded_greens):
                    result[f'相位{phase_idx+1}绿灯时长'] = green_time
                    
                results.append(result)
                print(f"{data[0]['intersection_name']} ({'工作日' if data[0]['is_weekday'] else '节假日'}) 优化完成")
            except Exception as e:
                print(f"优化 {data[0]['intersection_name']} 时出错: {e}")
                
                            # 收集结果
                for future, data in futures:
                    try:
                        optimal_greens, min_delay, cycle = future.result()
                        # 对最优绿灯时间和最小延迟进行四舍五入取整
                        rounded_greens = [round(green) for green in optimal_greens]
                        rounded_delay = round(min_delay)
                        
                        # 构建结果字典，将绿灯时长按相位分开
                        result = {
                            '交叉口': data['intersection_name'],
                            '是否为工作日': data['is_weekday'],
                            '时段': data['time_period'],
                            '最小延迟': rounded_delay,
                            '周期': round(cycle)
                        }
                        
                        # 添加各相位绿灯时长
                        for phase_idx, green_time in enumerate(rounded_greens):
                            result[f'相位{phase_idx+1}绿灯时长'] = green_time
                            
                        results.append(result)
                        print(f"{data['intersection_name']} ({'工作日' if data['is_weekday'] else '节假日'}) 优化完成")
                    except Exception as e:
                        print(f"优化 {data['intersection_name']} 时出错: {e}")

        # 保存结果
        results_df = pd.DataFrame(results)
        results_df.to_csv('./问题2支撑材料/traffic_signal_optimization_results.csv', index=False, encoding='gbk')
        print("所有优化完成，结果已保存")

        return results_df


    main()

早停机制触发，在第7代停止
早停机制触发，在第7代停止
环东路-纬中路 (工作日) 优化完成
环东路-纬中路 (节假日) 优化完成
环西路-纬中路 (工作日) 优化完成
环西路-纬中路 (节假日) 优化完成
经二路-纬中路 (工作日) 优化完成
经二路-纬中路 (节假日) 优化完成
经三路-纬中路 (工作日) 优化完成
经三路-纬中路 (节假日) 优化完成
经四路-纬中路 (工作日) 优化完成
经四路-纬中路 (节假日) 优化完成
经五路-纬中路 (工作日) 优化完成
经五路-纬中路 (节假日) 优化完成
经一路-纬中路 (工作日) 优化完成
经一路-纬中路 (节假日) 优化完成
经中路-环北路 (工作日) 优化完成
经中路-环北路 (节假日) 优化完成
经中路-环南路 (工作日) 优化完成
经中路-环南路 (节假日) 优化完成
经中路-纬一路 (工作日) 优化完成
经中路-纬一路 (节假日) 优化完成
纬中路-景区出入口 (工作日) 优化完成
纬中路-景区出入口 (节假日) 优化完成
所有优化完成，结果已保存
